In [ ]:
import pandas as pd
from tqdm import tqdm
import datetime
from collections import defaultdict
from sql import SqlMiddleware

In [ ]:
sql = SqlMiddleware(password="Datathon-2022", db_password="1&n0D4o9")

In [ ]:
label_df = pd.read_csv('../data/label.csv')
raw_labevents_df = pd.read_csv('../data/raw_labevents.csv')
raw_patientsinfo_df = pd.read_csv('../data/raw_patientsinfo.csv')
raw_procedureevents_df = pd.read_csv('../data/raw_procedureevents.csv')
raw_chatted_event_df = pd.read_csv('../data/raw_chart_event.csv')

In [ ]:
main_df = label_df

In [ ]:
# Age Gender
main_df = main_df.merge(raw_patientsinfo_df, on='subject_id', how='left')

In [15]:
rows = list()

for index, row in tqdm(list(main_df.iterrows())[:10000]):
    subject_id = row['subject_id']
    hadm_id = row['hadm_id']
    clabsi_label = row['clabsi']
    gender = row["gender"]
    age = row["age"]
    
    Invasive_table = raw_procedureevents_df[(raw_procedureevents_df["subject_id"] == subject_id) & (raw_procedureevents_df["hadm_id"] == hadm_id)]
    starttimes = [datetime.datetime.strptime(t_str, "%Y-%m-%d %H:%M:%S") for t_str in Invasive_table["starttime"]]
    endtimes = [datetime.datetime.strptime(t_str, "%Y-%m-%d %H:%M:%S") for t_str in Invasive_table["endtime"]]

    # Start time and End time
    start_time = sorted(starttimes)[0]
    end_time = sorted(endtimes)[-1] + datetime.timedelta(days=2)

    feature = dict()

    for label in ["Temperature", "Glucose", "Sodium", "Intubated", "Lactate", "White Blood Cells", "Platelet Count"]:
        s_time = start_time
        e_time = start_time + datetime.timedelta(days=1)

        event_result = raw_labevents_df[(raw_labevents_df["subject_id"] == subject_id) & (raw_labevents_df["hadm_id"] == hadm_id) & (raw_labevents_df["label"] == label)]
        event_list = list()
        for index, event_row in event_result.iterrows():
            event_list += [{
                "charttime": datetime.datetime.strptime(event_row["charttime"], "%Y-%m-%d %H:%M:%S"),
                "value": event_row["valuenum"]
            }]

        time_point = 0
        while e_time <= end_time:
            if time_point not in feature:
                feature[time_point] = dict()

            s_time += datetime.timedelta(days=1)
            e_time += datetime.timedelta(days=1)

            temp_list = list()
            for event in event_list:
                if event["charttime"] >= s_time and event["charttime"] < e_time:
                    temp_list += [float(event["value"])]

            feature[time_point][label]  = {
                "label": label,
                "time_point": time_point,
                "max": max(temp_list) if temp_list else None,
                "min": min(temp_list) if temp_list else None, 
                "average": (sum(temp_list) / len(temp_list)) if temp_list else None, 
            }
            time_point += 1

    # print(feature)
    
    
    for timepoint in feature:
        rows += [{
            "subject_id": subject_id,
            "hadm_id": hadm_id,
            "timepoint": timepoint,
            "clabsi": clabsi_label,
            "gender": gender,
            "age": age,
            "Temperature_max": feature[timepoint]["Temperature"]["max"],
            "Temperature_min": feature[timepoint]["Temperature"]["min"],
            "WBC_max": feature[timepoint]["White Blood Cells"]["max"],
            "WBC_min": feature[timepoint]["White Blood Cells"]["min"],
            "Glucose_max": feature[timepoint]["Glucose"]["max"],
            "Glucose_min": feature[timepoint]["Glucose"]["min"],
            "Sodium_max": feature[timepoint]["Sodium"]["max"],
            "Sodium_min": feature[timepoint]["Sodium"]["min"],
            "Intubated_max": feature[timepoint]["Intubated"]["max"],
            "Intubated_min": feature[timepoint]["Intubated"]["min"],
            "Lactate_max": feature[timepoint]["Lactate"]["max"],
            "Lactate_min": feature[timepoint]["Lactate"]["min"],
            "PLE_max": feature[timepoint]["Platelet Count"]["max"],
            "PLE_min": feature[timepoint]["Platelet Count"]["min"],
        }]
df = pd.DataFrame(rows)

100%|██████████| 20/20 [00:14<00:00,  1.37it/s]


In [16]:
df

,subject_id,hadm_id,timepoint,clabsi,gender,age,Temperature_max,Temperature_min,WBC_max,WBC_min,Glucose_max,Glucose_min,Sodium_max,Sodium_min,Intubated_max,Intubated_min,Lactate_max,Lactate_min,PLE_max,PLE_min
0,12147277,24186220,0,0,M,63,NaN,NaN,22.0,18.8,115.0,115.0,136.0,136.0,NaN,NaN,NaN,NaN,133.0,115.0
1,12147277,24186220,1,0,M,63,NaN,NaN,17.5,13.8,73.0,73.0,135.0,135.0,NaN,NaN,NaN,NaN,88.0,62.0
2,12147277,24186220,2,0,M,63,NaN,NaN,11.9,10.7,80.0,80.0,134.0,134.0,NaN,NaN,NaN,NaN,72.0,65.0
3,19934880,28186624,0,0,F,51,NaN,NaN,16.1,16.1,105.0,105.0,142.0,142.0,NaN,NaN,1.2,0.9,367.0,367.0
4,19934880,28186624,1,0,F,51,NaN,NaN,9.6,8.9,154.0,107.0,140.0,135.0,NaN,NaN,0.8,0.8,314.0,265.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,17490629,23424475,9,0,M,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,17490629,23424475,10,0,M,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,14265645,23447526,0,0,M,67,NaN,NaN,9.7,9.7,107.0,107.0,136.0,136.0,NaN,NaN,1.0,1.0,138.0,138.0
129,14265645,23447526,1,0,M,67,NaN,NaN,12.8,12.8,135.0,135.0,137.0,137.0,NaN,NaN,NaN,NaN,158.0,158.0
